In [8]:
import os
import cv2
import numpy as np
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report


In [11]:
import zipfile

zip_path = "/content/GTSRB (2).zip"
extract_path = "/content/GTSRB"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [12]:
DATA_ROOT = "/content/GTSRB/GTSRB"  # άλλαξέ το αν χρειάζεται

train_root = os.path.join(DATA_ROOT, "train_modified")
test_root  = os.path.join(DATA_ROOT, "test_modified")

print("Train root:", train_root)
print("Test root:", test_root)
print("Περιεχόμενα DATA_ROOT:", os.listdir(DATA_ROOT))


Train root: /content/GTSRB/GTSRB/train_modified
Test root: /content/GTSRB/GTSRB/test_modified
Περιεχόμενα DATA_ROOT: ['train_modified', 'test_modified']


In [14]:
import zipfile

zip_path = "/content/GTSRB (2).zip"
extract_path = "/content/GTSRB"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [15]:
def load_images_from_folder(root_dir):
    """
    root_dir: π.χ. '/content/GTSRB/GTSRB/train_modified'
    Επιστρέφει:
      - images: λίστα από grayscale εικόνες (numpy arrays)
      - labels: numpy array [N] με integer labels
      - class_names: λίστα με ονόματα κλάσεων (string)
    """
    images = []
    labels = []

    class_names = sorted([
        d for d in os.listdir(root_dir)
        if os.path.isdir(os.path.join(root_dir, d))
    ])
    label_map = {cls_name: idx for idx, cls_name in enumerate(class_names)}

    for cls_name in class_names:
        cls_dir = os.path.join(root_dir, cls_name)
        for fname in os.listdir(cls_dir):
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.ppm', '.bmp')):
                continue
            path = os.path.join(cls_dir, fname)
            img = cv2.imread(path)
            if img is None:
                continue
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img_gray)
            labels.append(label_map[cls_name])

    labels = np.array(labels, dtype=np.int32)
    return images, labels, class_names


train_images, train_labels, class_names = load_images_from_folder(train_root)
test_images,  test_labels,  _          = load_images_from_folder(test_root)

num_classes = len(class_names)

print("Train images:", len(train_images))
print("Test images:", len(test_images))
print("Classes:", num_classes)
print("Class names:", class_names[:10], "...")


Train images: 3977
Test images: 1983
Classes: 43
Class names: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17'] ...


In [16]:
def extract_sift_descriptors(images):
    """
    Επιστρέφει:
      - descriptors_per_image: λίστα, ένα array (N_i x 128) ανά εικόνα
      - all_descriptors: όλα τα descriptors σωρευτικά (M x 128)
    """
    sift = cv2.SIFT_create()
    descriptors_per_image = []
    all_desc_list = []

    for img in images:
        # Προαιρετικά: μπορείς να κάνεις resize για ταχύτητα
        # img = cv2.resize(img, (64, 64))

        keypoints, desc = sift.detectAndCompute(img, None)
        if desc is None:
            # Αν δεν βρει keypoints → βάλε έναν μηδενικό descriptor
            desc = np.zeros((1, 128), dtype=np.float32)
        descriptors_per_image.append(desc)
        all_desc_list.append(desc)

    all_descriptors = np.vstack(all_desc_list).astype(np.float32)
    return descriptors_per_image, all_descriptors


train_descs, all_train_descs = extract_sift_descriptors(train_images)
test_descs,  _              = extract_sift_descriptors(test_images)

print("Συνολικοί train descriptors:", all_train_descs.shape)


Συνολικοί train descriptors: (91459, 128)


In [18]:
def build_visual_vocabulary(all_descriptors, vocab_size, max_samples=100000):
    """
    all_descriptors: (M x D)
    vocab_size: αριθμός clusters (visual words)
    max_samples: προαιρετικό subsampling για ταχύτητα
    """
    if all_descriptors.shape[0] > max_samples:
        idx = np.random.choice(all_descriptors.shape[0], max_samples, replace=False)
        data = all_descriptors[idx]
    else:
        data = all_descriptors

    data = data.astype(np.float32)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.01)
    attempts = 3
    flags = cv2.KMEANS_PP_CENTERS

    compactness, labels, centers = cv2.kmeans(
        data,
        vocab_size,
        None,
        criteria,
        attempts,
        flags
    )

    return centers  # (vocab_size x D)


In [17]:
DATA_ROOT = "/content/GTSRB/GTSRB"  # άλλαξέ το αν χρειάζεται

train_root = os.path.join(DATA_ROOT, "train_modified")
test_root  = os.path.join(DATA_ROOT, "test_modified")

print("Train root:", train_root)
print("Test root:", test_root)
print("Περιεχόμενα DATA_ROOT:", os.listdir(DATA_ROOT))

Train root: /content/GTSRB/GTSRB/train_modified
Test root: /content/GTSRB/GTSRB/test_modified
Περιεχόμενα DATA_ROOT: ['train_modified', 'test_modified']


In [19]:
def compute_bovw_histograms(descriptors_per_image, vocabulary):
    """
    descriptors_per_image: λίστα από (N_i x D)
    vocabulary: (K x D)
    Επιστρέφει:
      - features: (N_images x K) BoVW histograms (L1-normalized)
    """
    vocab_size = vocabulary.shape[0]
    D = vocabulary.shape[1]
    vocab = vocabulary.astype(np.float32)

    features = []

    for desc in descriptors_per_image:
        desc = desc.astype(np.float32)
        hist = np.zeros(vocab_size, dtype=np.float32)

        # Για κάθε descriptor, βρίσκουμε το κοντινότερο visual word
        for d in desc:
            diff = vocab - d
            dist2 = np.sum(diff * diff, axis=1)
            idx = np.argmin(dist2)
            hist[idx] += 1.0

        # L1 normalization
        s = hist.sum()
        if s > 0:
            hist /= s

        features.append(hist)

    features = np.vstack(features).astype(np.float32)
    return features


In [20]:
def train_knn(train_features, train_labels, k):
    knn = cv2.ml.KNearest_create()
    knn.train(train_features, cv2.ml.ROW_SAMPLE, train_labels)
    return knn, k

def knn_predict(knn, k, samples):
    ret, results, neighbours, dist = knn.findNearest(samples, k)
    return results.flatten().astype(np.int32)


In [21]:
def train_ova_svm(train_features, train_labels, num_classes, C=1.0):
    """
    Εκπαιδεύει num_classes SVMs:
    για κάθε κλάση c:
        labels_binary = +1 για δείγματα της κλάσης c, -1 για τα υπόλοιπα
    """
    classifiers = []
    for c in range(num_classes):
        labels_binary = np.where(train_labels == c, 1, -1).astype(np.int32)

        svm = cv2.ml.SVM_create()
        svm.setType(cv2.ml.SVM_C_SVC)
        svm.setKernel(cv2.ml.SVM_LINEAR)  # όπως ζητά η εκφώνηση
        svm.setC(C)
        svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 1000, 1e-6))

        svm.train(train_features, cv2.ml.ROW_SAMPLE, labels_binary)
        classifiers.append(svm)

    return classifiers


def ova_svm_predict(classifiers, samples):
    """
    Για κάθε SVM παίρνουμε score (απόσταση από hyperplane) και διαλέγουμε την κλάση
    με το μεγαλύτερο score.
    """
    num_samples = samples.shape[0]
    num_classes = len(classifiers)
    scores = np.zeros((num_samples, num_classes), dtype=np.float32)

    for i, svm in enumerate(classifiers):
        # RAW_OUTPUT: signed distance από το hyperplane
        _, raw = svm.predict(samples, flags=cv2.ml.STAT_MODEL_RAW_OUTPUT)
        raw = raw.reshape(-1)
        # συχνά η OpenCV δίνει -distance, αντιστρέφουμε το πρόσημο
        scores[:, i] = -raw

    preds = np.argmax(scores, axis=1)
    return preds.astype(np.int32)


In [22]:
def evaluate_predictions(y_true, y_pred, class_names, title=""):
    if title:
        print("====", title, "====")
    acc = accuracy_score(y_true, y_pred)
    print("Συνολική ακρίβεια: {:.4f}".format(acc))
    print("\nΑκρίβεια ανά κλάση:")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    return acc


In [23]:
vocab_sizes = [50, 100, 200, 400, 800, 1500, 2000]
k_values = [1, 3, 5, 7, 9]

results = {}

for V in vocab_sizes:
    print("\n==============================")
    print(f"ΛΕΞΙΚΟ ΜΕΓΕΘΟΥΣ V = {V}")
    vocab = build_visual_vocabulary(all_train_descs, V)

    train_bovw = compute_bovw_histograms(train_descs, vocab)
    test_bovw  = compute_bovw_histograms(test_descs,  vocab)

    # ----- k-NN: εύρεση βέλτιστου k -----
    best_acc_knn = -1.0
    best_k = None

    for k in k_values:
        knn, _ = train_knn(train_bovw, train_labels, k)
        knn_preds = knn_predict(knn, k, test_bovw)
        acc_k = accuracy_score(test_labels, knn_preds)
        print(f"k-NN με k={k}: accuracy={acc_k:.4f}")

        if acc_k > best_acc_knn:
            best_acc_knn = acc_k
            best_k = k

    print(f"Βέλτιστο k για V={V}: {best_k} (accuracy={best_acc_knn:.4f})")

    # ----- One-vs-all SVM -----
    svm_classifiers = train_ova_svm(train_bovw, train_labels, num_classes)
    svm_preds = ova_svm_predict(svm_classifiers, test_bovw)
    acc_svm = accuracy_score(test_labels, svm_preds)
    print(f"SVM (one-vs-all, linear) για V={V}: accuracy={acc_svm:.4f}")

    # Αποθήκευση αποτελεσμάτων
    results[V] = {
        "best_k": best_k,
        "knn_best_acc": best_acc_knn,
        "svm_acc": acc_svm
    }

print("\n\n===== ΤΕΛΙΚΟΣ ΠΙΝΑΚΑΣ ΑΠΟΤΕΛΕΣΜΑΤΩΝ =====")
for V in vocab_sizes:
    vals = results[V]
    print(f"V={V}: k-NN -> k={vals['best_k']}, acc={vals['knn_best_acc']:.4f} | SVM acc={vals['svm_acc']:.4f}")



ΛΕΞΙΚΟ ΜΕΓΕΘΟΥΣ V = 50
k-NN με k=1: accuracy=0.2057
k-NN με k=3: accuracy=0.2103
k-NN με k=5: accuracy=0.2325
k-NN με k=7: accuracy=0.2310
k-NN με k=9: accuracy=0.2355
Βέλτιστο k για V=50: 9 (accuracy=0.2355)
SVM (one-vs-all, linear) για V=50: accuracy=0.1720

ΛΕΞΙΚΟ ΜΕΓΕΘΟΥΣ V = 100
k-NN με k=1: accuracy=0.2552
k-NN με k=3: accuracy=0.2557
k-NN με k=5: accuracy=0.2698
k-NN με k=7: accuracy=0.2763
k-NN με k=9: accuracy=0.2758
Βέλτιστο k για V=100: 7 (accuracy=0.2763)
SVM (one-vs-all, linear) για V=100: accuracy=0.3056

ΛΕΞΙΚΟ ΜΕΓΕΘΟΥΣ V = 200
k-NN με k=1: accuracy=0.2869
k-NN με k=3: accuracy=0.2824
k-NN με k=5: accuracy=0.2965
k-NN με k=7: accuracy=0.2960
k-NN με k=9: accuracy=0.2975
Βέλτιστο k για V=200: 9 (accuracy=0.2975)
SVM (one-vs-all, linear) για V=200: accuracy=0.3752

ΛΕΞΙΚΟ ΜΕΓΕΘΟΥΣ V = 400
k-NN με k=1: accuracy=0.3026
k-NN με k=3: accuracy=0.2915
k-NN με k=5: accuracy=0.3116
k-NN με k=7: accuracy=0.3182
k-NN με k=9: accuracy=0.3192
Βέλτιστο k για V=400: 9 (accuracy=0.3192)

In [24]:
best_V = 2000  # καλύτερο
vocab = build_visual_vocabulary(all_train_descs, best_V)
train_bovw = compute_bovw_histograms(train_descs, vocab)
test_bovw  = compute_bovw_histograms(test_descs,  vocab)

# k-NN με το βέλτιστο κ
best_k = results[best_V]["best_k"]
knn, _ = train_knn(train_bovw, train_labels, best_k)
knn_preds = knn_predict(knn, best_k, test_bovw)
evaluate_predictions(test_labels, knn_preds, class_names, title=f"k-NN (V={best_V}, k={best_k})")

# SVM αναλυτικά
svm_classifiers = train_ova_svm(train_bovw, train_labels, num_classes)
svm_preds = ova_svm_predict(svm_classifiers, test_bovw)
evaluate_predictions(test_labels, svm_preds, class_names, title=f"SVM one-vs-all (V={best_V})")


==== k-NN (V=2000, k=9) ====
Συνολική ακρίβεια: 0.3167

Ακρίβεια ανά κλάση:
              precision    recall  f1-score   support

           0     0.5000    0.1111    0.1818         9
           1     0.2092    0.4386    0.2833       114
          10     0.8824    0.5769    0.6977       104
          11     0.0886    0.7879    0.1593        66
          12     0.2654    0.3945    0.3173       109
          13     0.3421    0.4561    0.3910       114
          14     0.5227    0.5476    0.5349        42
          15     0.2000    0.0303    0.0526        33
          16     0.8000    0.1739    0.2857        23
          17     0.8125    0.2281    0.3562        57
          18     0.4800    0.1967    0.2791        61
          19     0.3636    0.4444    0.4000         9
           2     0.6327    0.5254    0.5741       118
          20     0.0000    0.0000    0.0000        14
          21     0.0370    0.0714    0.0488        14
          22     0.7500    0.3158    0.4444        19
     

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


==== SVM one-vs-all (V=2000) ====
Συνολική ακρίβεια: 0.5719

Ακρίβεια ανά κλάση:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         9
           1     0.6847    0.6667    0.6756       114
          10     0.6822    0.7019    0.6919       104
          11     0.8030    0.8030    0.8030        66
          12     0.4959    0.5505    0.5217       109
          13     0.5608    0.7281    0.6336       114
          14     0.7105    0.6429    0.6750        42
          15     0.5000    0.3333    0.4000        33
          16     0.6000    0.6522    0.6250        23
          17     0.5574    0.5965    0.5763        57
          18     0.6429    0.4426    0.5243        61
          19     0.8000    0.4444    0.5714         9
           2     0.7373    0.7373    0.7373       118
          20     0.5833    0.5000    0.5385        14
          21     0.0000    0.0000    0.0000        14
          22     0.6190    0.6842    0.6500        19


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.5718608169440242